In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [37]:
df=pd.read_csv("/content/drive/My Drive/AML_Assignment_2/IMDB Dataset.csv")

## Converting Sentimennts into 1s and 0s.

In [38]:
#sentiment count
df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})

## Denoising Text ie removing non-alphaneumeric characters and html tags.

In [39]:
import re
#Removing the html strips
def strip_html(text):
    p = re.compile(r'<.*?>')
    return p.sub(' ', text)

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text
#Apply function on review column
df['mod_review']=df['review'].apply(denoise_text)

In [40]:
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-Z0-9\s]'
    text=re.sub(pattern,' ',text)
    return text
#Apply function on review column
df['mod_review']=df['mod_review'].apply(remove_special_characters)

##Lemmatiztion and Removing Stopwords

In [43]:
stopword_list=nltk.corpus.stopwords.words('english')

In [42]:
from nltk.stem import WordNetLemmatizer

stop_words = set(stopwords.words("english")) 
lemmatizer = WordNetLemmatizer()


def lemmatize_text(text):
    text = text.lower()
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
    text = [word for word in text if not word in stop_words]
    text = " ".join(text)
    return text

df['mod_review'] = df['mod_review'].apply(lemmatize_text)

In [44]:
data =[]
for i in range(50000):
    temp = df['mod_review'][i].split()
    data.append(temp)
    

## Importing gensim and training Word2Vec from IMDB dataset

In [50]:
import gensim
model1 = gensim.models.Word2Vec(sentences = data, min_count = 1,size = 64,window = 4,workers = 5)
# model1.train(data,total_examples = len(data),epochs = 10)
words = list(model1.wv.vocab)

In [18]:
print("Vocab size : %d" % len(words))

Vocab size : 81980


In [55]:
model1.wv.most_similar('game')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('propagandish', 0.5994507074356079),
 ('football', 0.5986198782920837),
 ('n64', 0.5921552181243896),
 ('chess', 0.5819785594940186),
 ('baseball', 0.5565353631973267),
 ('goldeneye', 0.5437368750572205),
 ('nintendo', 0.5374946594238281),
 ('ps1', 0.5339474678039551),
 ('rugby', 0.5275295972824097),
 ('64', 0.5238086581230164)]

In [56]:
model1.wv.most_similar(positive = ['princess','boy'], negative = ['girl'],topn=1)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('bride', 0.7667683362960815)]

In [57]:
model1.wv.most_similar(positive = ['actor','woman'], negative = ['man'],topn=1)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('actress', 0.7171274423599243)]

In [53]:
model1.wv.most_similar(positive = ['game','football'], negative = ['foot'],topn=1)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('baseball', 0.707888126373291)]